2021.11.26 ~ 2022.02.28 => 주어진 데이터,
2022.03.04 ~ 13 그리고 03.18 ~ 20  => 검증 데이터

*최종 제출 방식))))))예보 데이터 시간 데이터 이용 => sum으로 일자 제출
분별 대체로 볼 때 0.2에서 0.4라고 한다면, 0.2 * 1440에서 0.4 * 1440 =>대략 300~500?

우선! 바운더리에 들어오면, 에너지 절감 타당한 모델을 만드는 게 핵심. 노가다 ㄴㄴ

*외부 데이터 방식)))))))관련 예보 데이터에 나와 있는 관측소만 이용해서 평균으로 보내서 하면 근거가 충분할 듯함
(주어진 게 그 관측소뿐이므로)

*주어진 데이터 방식))))열 공급량은 분=>시간 단위로 바뀔 때 합산해야함 (논리적으로 봤을 때, 시간 단위라면 분 단위의 누적이 되어야 맞는 거임. 의미적으로나 수치적으로나 둘 다.)
나머지 변수 => 일단 평균으로 (의미상,수치상 합산은 아닌듯..기온같은 경우..)

*공통))))outlier채우기, 중간에 없는 일자 뽑아서 외부,본 데이터 둘 다 삭제, 검증 데이터 3/15 삭제

*본 데이터12월 빠진 데이터 있음. (22,23,24), 외부 데이터 3월 15일 제거해야함

위는 따라서 외부 데이터에서 제거할 목록임

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor
import random
import xgboost as xgb

In [2]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf

from tensorflow.keras.layers import LSTM 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, SimpleRNN, GRU
import keras.backend as K 
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import SGD

In [3]:
import datetime

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout

# 데이터 로드

In [380]:
test1

,smart_farm.yy,smart_farm.mm,smart_farm.dd,smart_farm.hh,smart_farm.nn,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,...,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,2022,3,4,0,0,-4.0,2.0,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
1,2022,3,4,0,1,-4.1,2.0,76,0.4,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
2,2022,3,4,0,2,-4.1,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
3,2022,3,4,0,3,-4.1,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
4,2022,3,4,0,4,-4.2,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18715,2022,3,20,23,55,-3.0,6.2,61,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18716,2022,3,20,23,56,-3.2,6.2,61,0.0,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18717,2022,3,20,23,57,-3.2,6.1,61,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18718,2022,3,20,23,58,-3.3,6.1,61,0.4,346,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN


In [4]:
curr_dir = os.getcwd()
curr_dir

forders_tr = os.listdir(curr_dir+"\\data")
forders_tr

df = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
df = df.drop(df.columns[0], axis=1)
df

df2022 = df[df['smart_farm.yy']==2022]
df2022
df202203 = df2022[df2022['smart_farm.mm']==3]
df202203

train = df.iloc[:132480]
train = train.reset_index()
train = train.drop('index', axis=1)

test = df.iloc[132480:]
test = test[test['smart_farm.dd']!=15]        # 15일자 제외
test = test.reset_index()
test = test.drop('index', axis=1)
test

# train1 = train.dropna(axis=0)
train1 = train.interpolate(method ='linear')     
# train 결측치 채운 이유=> 시간 별 데이터 중간중간 결측돼서 이를 이용해서 일자 예측하려면 채워줘야하기 때문.
test1 = test.interpolate(method ='linear') 

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()
submission1             # "년도 월 일 시 분 => 년도 월 일"  예측

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,NaN,NaN
1,2022,3,5,NaN,NaN
2,2022,3,6,NaN,NaN
3,2022,3,7,NaN,NaN
4,2022,3,8,NaN,NaN
5,2022,3,9,NaN,NaN
6,2022,3,10,NaN,NaN
7,2022,3,11,NaN,NaN
8,2022,3,12,NaN,NaN
9,2022,3,13,NaN,NaN


In [3]:
train1[train1.columns[-1]].value_counts()

0.500000    28429
0.000000    25599
0.100000    23461
0.400000    11202
0.600000     9269
            ...  
0.344715        1
0.339024        1
0.333333        1
0.327642        1
0.400847        1
Name: smart_farm.heat_supply, Length: 507, dtype: int64

In [4]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132480 entries, 0 to 132479
Data columns (total 23 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   smart_farm.yy                               132480 non-null  int64  
 1   smart_farm.mm                               132480 non-null  int64  
 2   smart_farm.dd                               132480 non-null  int64  
 3   smart_farm.hh                               132480 non-null  int64  
 4   smart_farm.nn                               132480 non-null  int64  
 5   smart_farm.insolation                       132480 non-null  float64
 6   smart_farm.out_tmperature                   132480 non-null  float64
 7   smart_farm.out_humidity                     132480 non-null  int64  
 8   smart_farm.wind_speed                       132480 non-null  float64
 9   smart_farm.wind_direction                   132480 non-null  int64  
 

# 외부 데이터 넣기

In [342]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 해당 시간 당 지점별 평균 내기
soil

,지점,지점명,일시,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,119,수원,2021-11-26 00:00,4.5,25.9,24.1,26.6
1,119,수원,2021-11-26 01:00,4.5,25.9,24.1,26.6
2,119,수원,2021-11-26 02:00,4.6,25.9,24.1,26.6
3,119,수원,2021-11-26 03:00,4.6,25.9,24.1,26.6
4,119,수원,2021-11-26 04:00,4.5,25.9,24.1,26.6
...,...,...,...,...,...,...,...
28822,980,강정,2022-03-20 19:00,23.5,34.3,38.0,40.8
28823,980,강정,2022-03-20 20:00,23.6,34.3,38.0,40.8
28824,980,강정,2022-03-20 21:00,23.6,34.3,38.0,40.8
28825,980,강정,2022-03-20 22:00,23.6,34.8,38.8,40.8


In [343]:
stn = list(set(soil['지점'].values))
stn

[129, 258, 970, 972, 973, 974, 977, 978, 980, 119, 702]

In [344]:
# 인덱스 리셋, 지점/지점명/일시 컬럼 삭제 ==> 추후 일시 컬럼 복구(df간의 계산 위해)
soil_df = []
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)
    soil_df.append(df)
soil_df[0]

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,15.6,16.8,28.3,35.0
1,15.5,16.6,28.1,35.0
2,15.5,16.6,28.1,35.0
3,15.6,16.6,28.1,35.0
4,15.6,16.5,28.1,35.0
...,...,...,...,...
2488,20.6,23.2,31.6,32.9
2489,20.6,23.3,31.5,32.9
2490,20.6,23.3,31.6,32.9
2491,20.7,23.2,31.6,32.9


In [345]:
a = []
for i in range(len(soil_)):
    a.append(len(soil_[i]['일시'].values))     # 중간에 없는 일자의 관측소도 있음.(2710가 최대=>나머지는 0으로 채움.)
a

[2493, 2710, 2710, 2352, 2710, 2331, 2710, 2709, 2710, 2710, 2682]

In [346]:
# stn     # 129, 972, 974, 978, 702 => 전처리할 파일 _1

In [347]:
ind_1 = [index for index,value in enumerate(a) if value != max(a)]
ind_1

[0, 3, 5, 7, 10]

In [348]:
stn_1 = []
for i in range(len(ind_1)):
    stn_1.append(stn[ind_1[i]])
stn_1

[129, 972, 974, 978, 702]

In [349]:
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join('stn\\'+str(stn[i])+'.csv'), index=False)
    
## 임의로 비어 있는 부분 nan 표기 엑셀 vlookup이용해서
## 그 후 0으로 채우고 => 평균화

In [350]:
stn[ind_1[1]]

972

In [351]:
# 참고로, 앞부분 NAN은 안 중요 => 모두 0으로 채워도 됨. 뒷부분 열만 쓰므로
for i in range(len(stn_1)):
    df = pd.read_csv(os.path.join('stn\\'+str(stn_1[i])+'_1.csv'), encoding='cp949')
    df = df.drop(list(df.columns[2:8]), axis=1) 
    df = df.fillna(0)
    df = df.drop(['지점', '지점명', '일시.1'], axis=1)
    df.columns = soil_df[1].columns
    soil_df[ind_1[i]] = df

In [352]:
# 일자 저장
df = pd.read_csv(os.path.join('stn\\'+str(stn_1[0])+'_1.csv'), encoding='cp949')
day = df['일시.1'].values
day

array(['2021-11-26 0:00', '2021-11-26 1:00', '2021-11-26 2:00', ...,
       '2022-03-20 21:00', '2022-03-20 22:00', '2022-03-20 23:00'],
      dtype=object)

In [353]:
# 하나의 데이터 프레임으로(빈 값 하나 만들어서)
soil_df1 = soil_df[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df1[soil_df[0].columns[i]] = 0
soil_df1

for i in range(len(soil_df)):
    soil_df1 = soil_df1 + soil_df[i]
soil_df1 = soil_df1/len(soil_df)
soil_df1  

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,21.400000,22.190909,21.800000,24.372727
1,21.327273,22.172727,21.900000,24.381818
2,21.281818,22.127273,21.745455,24.372727
3,21.227273,22.109091,21.718182,24.372727
4,21.163636,22.072727,21.700000,24.363636
...,...,...,...,...
2705,20.218182,21.172727,19.836364,22.627273
2706,20.172727,21.109091,19.800000,22.627273
2707,20.190909,21.072727,19.809091,22.609091
2708,20.136364,21.045455,19.854545,22.618182


In [354]:
soil_df1['일자'] = day
soil_df1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 0:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 1:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 2:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 3:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 4:00
...,...,...,...,...,...
2705,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
2706,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
2707,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
2708,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [355]:
# 본 데이터12월 빠진 데이터 있음. (22,23,24)
# 외부 데이터 3월 15일 제거해야함

In [356]:
d = soil_df1[soil_df1['일자'] == '2021-12-22 0:00'].index[0]
d1 = soil_df1[soil_df1['일자'] == '2022-03-15 0:00'].index[0]
# print(d1)

for i in range(d, d+24*3):
    soil_df1 = soil_df1.drop(i,axis=0)
    
for i in range(d1, d1+24):
    soil_df1 = soil_df1.drop(i,axis=0)
    
# soil_df1.iloc[d:d+24*3]
# soil_df1.iloc[d1:d1+24]
soil_df1 = soil_df1.reset_index()
soil_df1 = soil_df1.drop('index', axis=1)
soil_df1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 0:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 1:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 2:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 3:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 4:00
...,...,...,...,...,...
2609,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
2610,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
2611,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
2612,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [394]:
d = soil_df1[soil_df1['일자'] == '2022-03-01 0:00'].index[0]
d1 = soil_df1[soil_df1['일자'] == '2022-03-04 0:00'].index[0]
soil_train = soil_df1.iloc[:d]
soil_test = soil_df1.iloc[d1:]

soil_test = soil_test.reset_index()
soil_test = soil_test.drop('index', axis=1)
soil_test

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 0:00
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 1:00
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 2:00
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 3:00
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 4:00
...,...,...,...,...,...
374,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
375,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
376,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
377,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [392]:
soil_train.to_csv('train_prep.csv', index=False)
soil_test.to_csv('test_prep.csv', index=False)

In [359]:
# soil_train.to_csv('train_prep.csv', index=False)
i = pd.date_range('2021-11-26', periods=2352, freq='1H')
i

DatetimeIndex(['2021-11-26 00:00:00', '2021-11-26 01:00:00',
               '2021-11-26 02:00:00', '2021-11-26 03:00:00',
               '2021-11-26 04:00:00', '2021-11-26 05:00:00',
               '2021-11-26 06:00:00', '2021-11-26 07:00:00',
               '2021-11-26 08:00:00', '2021-11-26 09:00:00',
               ...
               '2022-03-03 14:00:00', '2022-03-03 15:00:00',
               '2022-03-03 16:00:00', '2022-03-03 17:00:00',
               '2022-03-03 18:00:00', '2022-03-03 19:00:00',
               '2022-03-03 20:00:00', '2022-03-03 21:00:00',
               '2022-03-03 22:00:00', '2022-03-03 23:00:00'],
              dtype='datetime64[ns]', length=2352, freq='H')

# 외부 데이터에도 없는 날짜 채워줘야함_train

In [5]:
i = pd.date_range('2021-11-26', periods=2280, freq='1H')         #2352
i
day1 = pd.DataFrame(i)
day1.columns = ['일자']
day1
day1.to_csv('day.csv', index=False)

In [6]:
soil_train1 = pd.read_csv('train_prep_1.csv', encoding='cp949')
soil_train1
soil_train1 = soil_train1.drop(list(soil_train1.columns[:6]), axis=1) 
# soil_train1 = soil_train1.fillna(0)
soil_train1 = soil_train1.drop(['일자.1'], axis=1)
soil_train1.columns = soil_df[1].columns
soil_train1.drop(2280, axis=0, inplace=True)

NameError: name 'soil_df' is not defined

In [ ]:
soil_train1 = soil_train1.interpolate(method ='linear')     # interpolation 적용하고 특정 날짜를 빼야 의미가 상통(전처리=>다시 날짜빼줘야함)
soil_train1

soil_train1['일자'] = i
soil_train1

In [ ]:
d = soil_train1[soil_train1['일자'] == '2021-12-22 0:00'].index[0]
# print(d1)

for i in range(d, d+24*3):
    soil_train1 = soil_train1.drop(i,axis=0)
    
soil_train1 = soil_train1.reset_index()
soil_train1 = soil_train1.drop('index', axis=1)
soil_train1

In [369]:
## 외부데이터 날짜에 조차 아예 없는 값이 있다
# 11/26~30, 12/31,1/31,2/28,3/1~3/3 => 95(5+31+31+28) => 총 95번 동안 0~23 채워서 full date하면 됨 => interpolation 결측치 채움(엑셀)
95*24 - 3*24

2208

# 외부 데이터에도 없는 날짜 채워줘야함_test

In [ ]:
ii = pd.date_range('2022-03-04', periods=17*24, freq='1H')         #2352
ii
day1 = pd.DataFrame(ii)
day1.columns = ['일자']
day1
# day1.to_csv('day_test.csv', index=False)
day1

In [ ]:
soil_test1 = pd.read_csv('test_prep_1.csv', encoding='cp949')
soil_test1
soil_test1 = soil_test1.drop(list(soil_test1.columns[:6]), axis=1) 
# soil_train1 = soil_train1.fillna(0)
soil_test1 = soil_test1.drop(['일자.1'], axis=1)
soil_test1.columns = soil_df[1].columns
# soil_test1.drop(2280, axis=0, inplace=True)
soil_test1

In [ ]:
soil_test1 = soil_test1.interpolate(method ='linear')     # interpolation 적용하고 특정 날짜를 빼야 의미가 상통(전처리=>다시 날짜빼줘야함)
soil_test1

soil_test1['일자'] = ii
soil_test1

In [ ]:
d = soil_test1[soil_test1['일자'] == '2022-03-14 0:00'].index[0]
soil_test1.iloc[d:d+24*4]

for i in range(d, d+24*4):
    soil_test1 = soil_test1.drop(i,axis=0)
    
soil_test1 = soil_test1.reset_index()
soil_test1 = soil_test1.drop('index', axis=1)
soil_test1

In [ ]:
#### 외부데이터로 뭉친 train,test 따로 저장 => 드라이브

# 본 데이터 시간별로 만들기

In [ ]:
col = ['smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']

col

In [328]:
train1

,smart_farm.yy,smart_farm.mm,smart_farm.dd,smart_farm.hh,smart_farm.nn,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,...,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,2021,11,26,0,0,-2.7,5.8,67,0.0,325,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
1,2021,11,26,0,1,-2.9,5.8,67,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
2,2021,11,26,0,2,-3.0,5.7,67,0.9,105,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.9
3,2021,11,26,0,3,-3.0,5.7,67,1.3,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
4,2021,11,26,0,4,-2.8,5.7,67,0.9,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132475,2022,2,28,23,55,-2.7,4.1,51,0.0,57,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.9
132476,2022,2,28,23,56,-2.7,4.1,52,0.4,57,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
132477,2022,2,28,23,57,-2.7,4.1,52,0.4,40,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
132478,2022,2,28,23,58,-2.6,4.1,52,0.4,36,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8


In [255]:
132480/60

2208.0

In [256]:
train_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(train1)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(train1[col[k]][60*i:60*(i+1)]))
    train_[col[k]] = h

    
h = []
for i in range(int(len(train1)/60)):
    h.append(np.mean(train1['smart_farm.heat_supply'][60*i:60*(i+1)]))
train_['smart_farm.heat_supply'] = h
train_        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]


,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.600000,28.0,18.000,0.780000
1,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.650000,28.0,18.000,0.811667
2,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.683333,28.0,18.000,0.750000
3,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.750000,28.0,18.000,0.766667
4,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.783333,28.0,18.000,0.516667
...,...,...,...,...,...,...,...,...,...,...,...
2203,0.0,61.75,0.0,0.0,0.0,0.0,1.000000,0.000000,28.0,16.750,0.100000
2204,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.000000,28.0,15.140,0.100000
2205,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.150000,28.0,15.390,0.246667
2206,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.833333,28.0,17.235,0.608333


In [419]:
train_real = pd.concat([soil_train1, train_], axis=1)
train_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 00:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.600000,28.0,18.000,0.780000
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 01:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.650000,28.0,18.000,0.811667
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 02:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.683333,28.0,18.000,0.750000
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 03:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.750000,28.0,18.000,0.766667
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 04:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.783333,28.0,18.000,0.516667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,12.381818,14.727273,13.590909,17.200000,2022-02-28 19:00:00,0.0,61.75,0.0,0.0,0.0,0.0,1.000000,0.000000,28.0,16.750,0.100000
2204,12.318182,14.772727,13.609091,17.209091,2022-02-28 20:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.000000,28.0,15.140,0.100000
2205,12.290909,14.790909,13.618182,17.209091,2022-02-28 21:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.150000,28.0,15.390,0.246667
2206,12.227273,14.727273,13.627273,17.209091,2022-02-28 22:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.833333,28.0,17.235,0.608333


In [420]:
# train_real.to_csv('train_real.csv', index=False)

# 검증 데이터 일자별 + 외부 셋 넣기

In [371]:
submission1

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,NaN,NaN
1,2022,3,5,NaN,NaN
2,2022,3,6,NaN,NaN
3,2022,3,7,NaN,NaN
4,2022,3,8,NaN,NaN
5,2022,3,9,NaN,NaN
6,2022,3,10,NaN,NaN
7,2022,3,11,NaN,NaN
8,2022,3,12,NaN,NaN
9,2022,3,13,NaN,NaN


In [404]:
test_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(test1)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(test1[col[k]][60*i:60*(i+1)]))
    test_[col[k]] = h

    
h = []
for i in range(int(len(test1)/60)):
    h.append(np.mean(test1['smart_farm.heat_supply'][60*i:60*(i+1)]))
test_['smart_farm.heat_supply'] = h
test_        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 31.24it/s]


,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
307,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


In [421]:
test_real = pd.concat([soil_test1, test_], axis=1)
test_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


In [422]:
# test_real.to_csv('test_real.csv', index=False)

# Model1

앞서 전처리된 train_real, test_real dataset 가져옴(제어장치 컬럼 포함된)

<순서>
주어진 데이터(21.11.26~22.02.28까지의 연속 데이터) 

22.03.01~22.03.20 (총20일간) 데이터 예측 => 이렇게 하면 구조상 문제 없음

원하는 값 뽑아낼 수 있게

1) 한꺼번에 24*20개씩 예측하도록 => 구조상 복잡함..

2) 한개씩(시간 단위로, 딱 한개씩, 일자 한개씩 X) 예측하도록 =>train,test,validation 다시 나눠야됨

In [4]:
train_real = pd.read_csv('train_real.csv')
test_real = pd.read_csv('test_real.csv')

In [9]:
train_real_s = pd.read_csv('train_real_sum.csv')
test_real_s = pd.read_csv('test_real_sum.csv')

In [29]:
train_real_s

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 00:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.600000,28.0,18.000,46.8
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 01:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.650000,28.0,18.000,48.7
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 02:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.683333,28.0,18.000,45.0
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 03:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.750000,28.0,18.000,46.0
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 04:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.783333,28.0,18.000,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,12.381818,14.727273,13.590909,17.200000,2022-02-28 19:00:00,0.0,61.75,0.0,0.0,0.0,0.0,1.000000,0.000000,28.0,16.750,6.0
2204,12.318182,14.772727,13.609091,17.209091,2022-02-28 20:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.000000,28.0,15.140,6.0
2205,12.290909,14.790909,13.618182,17.209091,2022-02-28 21:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.150000,28.0,15.390,14.8
2206,12.227273,14.727273,13.627273,17.209091,2022-02-28 22:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.833333,28.0,17.235,36.5


In [28]:
test_real_s

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,0.0
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,0.0
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,0.0
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,0.0
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,0.0
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,0.0
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,0.0
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,0.0


In [10]:
## 20일치 한 번에 예측(덩어리 잡고 => 기준: 검증셋 일수만큼)

X = train_real_s.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = train_real_s[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)   # shuffle은 안 됨. 시간적순서라

validation = test_real_s.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)




# scaler = MinMaxScaler()
# training_data = scaler.fit_transform(training_data)
# training_data
# test_data = scaler.fit_transform(test_data)
# test_data



x_train = []
y_train = []

for i in range(24*13, training_data.shape[0]):
  x_train.append(training_data[i-24*13:i])
  y_train.append(training_data.iloc[i,-1])
#   y_train.append(training_data.iloc[i:i+20, -1].values)
#   y_train.append(training_data[i, 0])
    
    
x_test = []
y_test = []

for i in range(24*13, test_data.shape[0]): # 롤링 기법에서 쓰이듯이 윈도우 =1
  x_test.append(test_data[i-24*13:i])
#   y_test.append(test_data.iloc[i:i+20, -1].values)     # [i,0]
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



# x_validation = []
# for i in range(24*13, validation.shape[0]): # 롤링 기법에서 쓰이듯이 윈도우 =1
#   x_validation.append(validation[i-24*13:i])
# #   y_test.append(.iloc[i, -1])     # [i,0]
# x_validation = np.array(x_validation)


# x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
print(x_train.shape) # 2188,20,10 => data수, 덩어리단위, 컬럼수
print(y_train.shape) # 2188, => data수, 타겟 값(heat_supply_day)
print(x_test.shape)
print(y_test.shape)
x_train[0]

regression_GRU = Sequential()
regression_GRU.add(GRU(units=50, activation="relu", return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2])))
regression_GRU.add(Dropout(0.2))

regression_GRU.add(GRU(units=60, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.3))

regression_GRU.add(GRU(units=80, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.4))

regression_GRU.add(GRU(units=120, activation="relu"))  # 624
regression_GRU.add(Dropout(0.5))

regression_GRU.add(Dense(units = 20*24))     # 1

regression_GRU.compile(optimizer='adam', loss='mean_squared_error')

GRU_history = regression_GRU.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 시계열 데이터는 시간별 데이터프레임을 통으로 넣고 돌리는 것

(1233, 312, 10)
(1233,)
(351, 312, 10)
(351,)
Epoch 1/10
39/39 [==============================] - 69s 1s/step - loss: 127.6795 - val_loss: 41.7504
Epoch 2/10
39/39 [==============================] - 56s 1s/step - loss: 17.3692 - val_loss: 29.7426
Epoch 3/10
39/39 [==============================] - 53s 1s/step - loss: 11.4039 - val_loss: 12.4248
Epoch 4/10
39/39 [==============================] - 52s 1s/step - loss: 9.2577 - val_loss: 12.4343
Epoch 5/10
39/39 [==============================] - 52s 1s/step - loss: 7.9298 - val_loss: 10.5438
Epoch 6/10
39/39 [==============================] - 53s 1s/step - loss: 6.9613 - val_loss: 11.3880
Epoch 7/10
39/39 [==============================] - 54s 1s/step - loss: 6.7406 - val_loss: 7.9502
Epoch 8/10
39/39 [==============================] - 54s 1s/step - loss: 6.5114 - val_loss: 8.3163
Epoch 9/10
39/39 [==============================] - 58s 1s/step - loss: 5.8765 - val_loss: 4.8332
Epoch 10/10
39/39 [==============================] - 56s 1s/st

In [19]:
regression_GRU.save('gru_1.h5')
model1 = load_model('gru_1.h5')
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 312, 50)           9300      
                                                                 
 dropout (Dropout)           (None, 312, 50)           0         
                                                                 
 gru_1 (GRU)                 (None, 312, 60)           20160     
                                                                 
 dropout_1 (Dropout)         (None, 312, 60)           0         
                                                                 
 gru_2 (GRU)                 (None, 312, 80)           34080     
                                                                 
 dropout_2 (Dropout)         (None, 312, 80)           0         
                                                                 
 gru_3 (GRU)                 (None, 120)               7

In [11]:
x_validation =[]
x_validation.append(validation)
x_validation = np.array(x_validation)
x_validation.shape
# x_validation = []
# for i in range(24*13, validation.shape[0]): # 롤링 기법에서 쓰이듯이 윈도우 =1
#   x_validation.append(validation[i-24*13:i])
# #   y_test.append(.iloc[i, -1])     # [i,0]
# x_validation = np.array(x_validation)

(1, 312, 10)

In [12]:
p = regression_GRU.predict(x_validation)    # 13일치 넣으면 -> 20일치 나오겠지
# p1 = scaler.inverse_transform(p)
p

1/1 [==============================] - 2s 2s/step


array([[15.637694 , 15.506315 , 15.692649 , 15.639039 , 15.655281 ,
        15.861987 , 15.845499 , 15.452514 , 15.919504 , 15.418943 ,
        15.779393 , 15.4936495, 15.546856 , 15.413088 , 15.52443  ,
        15.880141 , 15.71146  , 15.861456 , 15.902172 , 15.651595 ,
        15.857935 , 15.590788 , 15.43164  , 15.356221 , 15.645626 ,
        15.63178  , 15.589406 , 15.819371 , 15.829607 , 15.443138 ,
        15.985282 , 15.652071 , 15.420209 , 15.695361 , 15.204221 ,
        15.469384 , 15.806011 , 15.398718 , 15.4097185, 15.799467 ,
        15.31391  , 15.595229 , 15.533873 , 15.452428 , 15.426169 ,
        16.021143 , 15.966475 , 15.811765 , 15.692831 , 15.69044  ,
        15.996468 , 15.681581 , 15.805289 , 15.635656 , 15.956581 ,
        15.763405 , 15.498786 , 15.638503 , 15.002634 , 15.447119 ,
        15.640867 , 15.448147 , 15.594084 , 15.850952 , 15.905488 ,
        15.636702 , 15.39757  , 15.720643 , 15.442019 , 15.133006 ,
        15.749924 , 15.756031 , 15.831837 , 15.5

In [13]:
h1 = []
for i in range(int(len(p[0])/24)):      # 분 -> 시간 단위 바꾸기 위해
    h1.append(np.sum(p[0][24*i:24*(i+1)]))
h1     

[375.63025,
 374.92035,
 375.08472,
 374.51312,
 374.15115,
 375.64795,
 372.86118,
 376.4088,
 374.78125,
 374.9702,
 375.24908,
 376.89618,
 374.44934,
 373.13214,
 374.58307,
 376.47717,
 375.86957,
 375.3113,
 374.82724,
 375.32712]

In [14]:
# 4~13, 18~20
h1.insert(0,0)     # 한 번만 클릭!
h1

[0,
 375.63025,
 374.92035,
 375.08472,
 374.51312,
 374.15115,
 375.64795,
 372.86118,
 376.4088,
 374.78125,
 374.9702,
 375.24908,
 376.89618,
 374.44934,
 373.13214,
 374.58307,
 376.47717,
 375.86957,
 375.3113,
 374.82724,
 375.32712]

In [15]:
h11 = h1[4:14] + h1[18:21]
h11

[374.51312,
 374.15115,
 375.64795,
 372.86118,
 376.4088,
 374.78125,
 374.9702,
 375.24908,
 376.89618,
 374.44934,
 375.3113,
 374.82724,
 375.32712]

In [16]:
submission1['heat_supply_day1'] = h11
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,374.513123,NaN
1,2022,3,5,374.151154,NaN
2,2022,3,6,375.647949,NaN
3,2022,3,7,372.861176,NaN
4,2022,3,8,376.408813,NaN
5,2022,3,9,374.781250,NaN
6,2022,3,10,374.970215,NaN
7,2022,3,11,375.249084,NaN
8,2022,3,12,376.896179,NaN
9,2022,3,13,374.449341,NaN


# Model2

In [20]:
# 20일치 한 번에 예측(덩어리 잡고 => 기준: 검증셋 일수만큼)

X = train_real_s.drop(['일자','smart_farm.heat_supply'], axis=1)
y = train_real_s[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)   # shuffle은 안 됨. 시간적순서라

validation = test_real_s.drop(['일자','smart_farm.heat_supply'], axis=1)




# scaler = MinMaxScaler()
# training_data = scaler.fit_transform(training_data)
# training_data
# test_data = scaler.fit_transform(test_data)
# test_data



x_train = []
y_train = []

for i in range(24*13, training_data.shape[0]):
  x_train.append(training_data[i-24*13:i])
  y_train.append(training_data.iloc[i,-1])
#   y_train.append(training_data.iloc[i:i+20, -1].values)
#   y_train.append(training_data[i, 0])
    
    
x_test = []
y_test = []

for i in range(24*13, test_data.shape[0]): # 롤링 기법에서 쓰이듯이 윈도우 =1
  x_test.append(test_data[i-24*13:i])
#   y_test.append(test_data.iloc[i:i+20, -1].values)     # [i,0]
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



# x_validation = []
# for i in range(24*13, validation.shape[0]): # 롤링 기법에서 쓰이듯이 윈도우 =1
#   x_validation.append(validation[i-24*13:i])
# #   y_test.append(.iloc[i, -1])     # [i,0]
# x_validation = np.array(x_validation)


# x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
print(x_train.shape) # 2188,20,10 => data수, 덩어리단위, 컬럼수
print(y_train.shape) # 2188, => data수, 타겟 값(heat_supply_day)
print(x_test.shape)
print(y_test.shape)
x_train[0]

regression_GRU = Sequential()
regression_GRU.add(GRU(units=50, activation="relu", return_sequences=True, input_shape = (x_train.shape[1], x_train.shape[2])))
regression_GRU.add(Dropout(0.2))

regression_GRU.add(GRU(units=60, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.3))

regression_GRU.add(GRU(units=80, activation="relu", return_sequences=True))
regression_GRU.add(Dropout(0.4))

regression_GRU.add(GRU(units=120, activation="relu"))
regression_GRU.add(Dropout(0.5))

regression_GRU.add(Dense(units = 20*24))     # 1

regression_GRU.compile(optimizer='adam', loss='mean_squared_error')

GRU_history = regression_GRU.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 시계열 데이터는 시간별 데이터프레임을 통으로 넣고 돌리는 것

(1233, 312, 14)
(1233,)
(351, 312, 14)
(351,)
Epoch 1/10
39/39 [==============================] - 66s 1s/step - loss: 132.1589 - val_loss: 60.5097
Epoch 2/10
39/39 [==============================] - 53s 1s/step - loss: 18.1619 - val_loss: 51.3452
Epoch 3/10
39/39 [==============================] - 55s 1s/step - loss: 11.5949 - val_loss: 38.8363
Epoch 4/10
39/39 [==============================] - 48s 1s/step - loss: 9.6303 - val_loss: 32.0432
Epoch 5/10
39/39 [==============================] - 48s 1s/step - loss: 7.8399 - val_loss: 36.7632
Epoch 6/10
39/39 [==============================] - 48s 1s/step - loss: 7.1077 - val_loss: 43.0833
Epoch 7/10
39/39 [==============================] - 49s 1s/step - loss: 6.7834 - val_loss: 27.1991
Epoch 8/10
39/39 [==============================] - 49s 1s/step - loss: 6.1473 - val_loss: 21.9566
Epoch 9/10
39/39 [==============================] - 48s 1s/step - loss: 5.7568 - val_loss: 17.6906
Epoch 10/10
39/39 [==============================] - 48s 1s

In [21]:
regression_GRU.save('gru_2.h5')
model2 = load_model('gru_2.h5')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 312, 50)           9900      
                                                                 
 dropout_4 (Dropout)         (None, 312, 50)           0         
                                                                 
 gru_5 (GRU)                 (None, 312, 60)           20160     
                                                                 
 dropout_5 (Dropout)         (None, 312, 60)           0         
                                                                 
 gru_6 (GRU)                 (None, 312, 80)           34080     
                                                                 
 dropout_6 (Dropout)         (None, 312, 80)           0         
                                                                 
 gru_7 (GRU)                 (None, 120)              

In [22]:
x_validation =[]
x_validation.append(validation)
x_validation = np.array(x_validation)
x_validation.shape

(1, 312, 14)

In [23]:
p = regression_GRU.predict(x_validation)    # 13일치 넣으면 -> 20일치 나오겠지
# p1 = scaler.inverse_transform(p)
p

1/1 [==============================] - 2s 2s/step


array([[13.223368 , 13.246416 , 13.036114 , 12.912565 , 13.233494 ,
        13.354939 , 13.324782 , 13.154515 , 13.142447 , 12.919286 ,
        13.133765 , 13.252071 , 12.4957   , 12.923195 , 13.011916 ,
        12.983168 , 12.865072 , 13.4782295, 13.429452 , 13.052145 ,
        13.176284 , 13.250567 , 13.429272 , 13.116541 , 13.040568 ,
        13.204465 , 13.076548 , 13.335673 , 13.128249 , 13.379821 ,
        12.868007 , 13.657447 , 12.768365 , 13.145463 , 13.010932 ,
        13.317737 , 12.933608 , 12.976679 , 13.282008 , 12.916668 ,
        13.368453 , 13.314016 , 12.933178 , 12.989068 , 12.998942 ,
        13.135946 , 13.313541 , 13.2893915, 13.229523 , 13.265921 ,
        12.820957 , 12.600663 , 12.949198 , 13.331152 , 13.012698 ,
        13.349991 , 13.061796 , 13.141171 , 12.697944 , 13.140738 ,
        13.036204 , 13.294289 , 13.023832 , 13.265479 , 13.02133  ,
        13.498373 , 13.212802 , 12.821931 , 13.233261 , 13.242866 ,
        12.895404 , 13.164656 , 13.082776 , 12.6

In [24]:
h2 = []
for i in range(int(len(p[0])/24)):      # 분 -> 시간 단위 바꾸기 위해
    h2.append(np.sum(p[0][24*i:24*(i+1)]))
h2     

# 4~13, 18~20
h2.insert(0,0)     # 한 번만 클릭!
h2

h22 = h2[4:14] + h2[18:21]
h22

[314.03473,
 313.91385,
 318.50357,
 314.69647,
 315.08963,
 315.04385,
 311.41092,
 313.15616,
 314.4829,
 312.83475,
 314.7453,
 314.43628,
 316.70038]

In [25]:
submission1['heat_supply_day2'] = h22
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,374.513123,314.034729
1,2022,3,5,374.151154,313.913849
2,2022,3,6,375.647949,318.503571
3,2022,3,7,372.861176,314.696472
4,2022,3,8,376.408813,315.089630
5,2022,3,9,374.781250,315.043854
6,2022,3,10,374.970215,311.410919
7,2022,3,11,375.249084,313.156158
8,2022,3,12,376.896179,314.482910
9,2022,3,13,374.449341,312.834747


In [27]:
submission1.to_csv('220224_soil_timeseries_sum.csv', index=False)    # 3등

In [ ]:
### 와 눈에 띄게 줄었네~

In [ ]:
## 하긴 열 공급량이 같은 달인데 큰 차이 없는게 이상하진 않음!!

In [ ]:
## 여기서 더 건들지 말고 !!  하이퍼파라미터만 조금 더 손봐서(epoch, best weight)